# Building an AI Film Analysis Agent

## Business Objective

This notebook guides you through building an AI agent specialized in film analysis using Amazon Bedrock (high level workflow of Bedrock Agent framework below). The agent you develop will combine video analysis with film knowledge base and celebrity recognition capabilities to automatically identify and analyze film content. Media companies, which have long struggled with processing and identifying film content across their vast libraries, can leverage these agents to automate traditionally manual tasks. What once took hours of human review—identifying clips, recognizing actors, and extracting metadata—can now be completed automatically. 

![Bedrock Agent](../../static/images/02-bedrock-agent.png)

### Main Steps

1. **Knowledge Base Creation:** You'll build a film information knowledge base by uploading data to S3 and configuring vector search. Once synchronized, this database enables your agent to accurately match and identify video content.

2. **Film Agent Development:** You'll create the AI agent in Amazon Bedrock, configure it for film analysis, and connect it to your knowledge base. The agent uses foundation models to analyze content and match clips to source films.

3. **Celebrity Detection Integration:** You'll add celebrity detection by integrating Lambda and DynamoDB, enabling your agent to recognize actors and retrieve their role information. This helps verify film identification through cast appearances.

4. **Testing and Validation:** You'll test the agent's core functions to ensures accurate film identification through both content and cast analysis.

<div class="alert-warning">
    <b>Warning:</b>
    <p>
    1) please make sure to run <b>00-prerequisites.ipynb</b> to properly setup all the packages.
    </p>
</div>

## Initialize the parameters

In [ ]:
import boto3
import json
import uuid
import sagemaker
import uuid
from IPython.display import Video, display, Markdown, JSON

boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)
if sess.default_bucket():
    bucket = sess.default_bucket()
else:
    bucket = "<YOUR-BUCKET-NAME>" # Provide your own bucket
region = sess.boto_region_name
prefix = "film-agent-bedrock"

# set up S3 client
s3_client = boto_session.client('s3')

# access account id
sts_client = boto_session.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

### Analyze the video and video extraction

The video you will use is  Meridian(2016), a mystery film from [Netflix](https://opencontent.netflix.com/) - This content is available under the [Creative Commons Attribution 4.0 International Public License](https://creativecommons.org/licenses/by/4.0/legalcode)

The video analysis metadata is generated by [Bedrock Data Automation (BDA)](https://docs.aws.amazon.com/bedrock/latest/userguide/bda.html), a managed video understanding feature in Bedrock. If you are interested to learn how the analaysis is generated, please visit [Lab: Media Analysis Uisng BDA](../../1-media-analysis-using-bda/01-extract-analyze-a-movie.ipynb).

In [ ]:
video_file = 'NetflixMeridian.mp4'
!curl "https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/Netflix_Open_Content_Meridian.mp4" --output NetflixMeridian.mp4

Upload video to S3 for later steps

In [ ]:
s3_key = f"{prefix}/{video_file}"

s3_client.upload_file(video_file, bucket, s3_key)
film_video_s3_path = f"s3://{bucket}/{s3_key}"

In [ ]:
# Preview video and the metadata
display(Video(video_file))

display(Markdown("### Video Analysis Metadata:"))
with open('video_analysis.json', 'r') as file:
    video_analysis = json.load(file)
display(JSON(video_analysis))

## Setup Film Agent

Now that we have our video analysis results, let's create an AI agent using Amazon Bedrock Agents. Bedrock Agents provide a powerful framework for building AI applications that can understand natural language, process information, and take actions using foundation models.

In our case, we'll create an agent specialized in film analysis that can:
- Process video content analysis
- Search through film databases
- Recognize celebrities in footage
- Combine multiple sources of information to identify films

### Why Use an Agent?
Agents excel at orchestrating complex tasks that require multiple steps and different types of analysis. For film identification, our agent will:
1. Analyze the video content we extracted with BDA
2. Query a knowledge base of film information
3. Use celebrity recognition to identify actors
4. Combine these insights to make accurate film identifications

This automated approach helps media companies process content more efficiently than manual review while maintaining high accuracy.

In the following sections, we'll:
- Initialize the AWS environment for our agent
- Configure the agent with specific film analysis instructions
- Set up foundation models for the agent to use

For more information about Amazon Bedrock Agents and their capabilities, see the [Amazon Bedrock Agents documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html).

Let's start by setting up the necessary AWS clients and configurations:

In [ ]:
import os
import time
import sys

account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

bedrock_client = boto_session.client('bedrock-runtime', region)

agent_foundation_model = [
    'global.anthropic.claude-sonnet-4-20250514-v1:0',
]

### Configure Resource Names

Before creating our agent, we need to set up unique identifiers for all the AWS resources we'll use. Following AWS naming best practices, we'll create standardized names that clearly identify our resources while maintaining consistency across our solution.

This standardized naming approach helps us:
- Easily identify related resources
- Maintain clear resource organization
- Enable proper resource management
- Support future scaling of our solution

Let's define these resource names:

In [ ]:
# Configure agent, lambda name using region and account id for uniqueness
agent_name = f"f-agent-{agent_suffix}"
lambda_name = f"fn-f-agent-{agent_suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

# Configure knowledge base resource names
knowledge_base_name = f'{agent_name}-kb'
suffix = f"{region}-{account_id}"
knowledge_base_description = "KB containing information of all the films"
bucket_name = f'{agent_name}-{suffix}'

# Configure dynamoDB table for cast memeber database 
cast_table = f'cast-table-{agent_suffix}'
cast_pk = 'id'
cast_sk = 'name'
env_args = [cast_table, cast_pk, cast_sk]

### Importing helper functions

On following section, you will import `bedrock_agent_helper.py` and `knowledge_base_helper`. Those files contain helper classes totally focused on make labs experience smoothly. All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Synchronize Knowledge Base (KB)

For our film analysis agent to effectively identify movies, it needs access to accurate film information. Amazon Bedrock Knowledge Bases provides a fully managed solution that allows our agent to search and retrieve film details using natural language processing. 

### Why Use a KB?
Our agent needs to match video content with specific films by analyzing semantic similarities between extracted metadata and film documentation, comparing plots, cast details, production information, and scene descriptions. These elements are evaluated against actual film records to identify precise matches.

The knowledge base enables this capability by providing embedded film data that supports rapid searching, processes natural language queries, and delivers contextually relevant responses. This structured database serves as the foundation for accurate and efficient content identification.

### What We'll Build
In this section, we'll:
1. Create a knowledge base specialized for film information
2. Upload our film data to S3
3. Configure vector search capabilities for efficient matching
4. Synchronize the data to make it available to our agent

<div class="alert-warning">
    <b>Warning:</b> This process may take up to 5 minutes.
</div>

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Upload Film Documents to S3

Before our knowledge base can be used, we need to upload our film information to S3. We'll use the AWS S3 sync command to efficiently transfer our film documents to the cloud.

You will sync our film documents to the S3 bucket we created earlier.

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

films_folder = "films"

!aws s3 sync {films_folder} s3://{bucket_name}/

Now you will synchronize the uploaded film data with the knowledge base. Amazon Bedrock will:
- Generate vector embeddings from our film documents
- Create semantic indexes for efficient retrieval
- Prepare the data for RAG (Retrieval Augmented Generation)

This synchronization transforms our text-based film information into a format optimized for AI-powered searching. Once complete, your agent will be able to quickly find and retrieve relevant film information based on video content analysis.

Let's initiate the synchronization:

In [ ]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

KB id and ARN is the unique identifier to connecting your agent to the knowledge base. You will also attach an instruction, so the agent know when to use the KB.

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when you need to look up title, director, and plot of a film"""
}

## Creating Agent

And now let's create and configure the film agent in Amazon Bedrock. 

Amazon Bedrock Agents offers you the ability to build and configure autonomous agents in your application.  With agents, you can automate tasks for your customers and answer questions for them. 

In this particular task we will create an instruction to analyze film videos or deriative of film videos and match it to the correct film title and identify key cast memebers.

In [ ]:
agent_description = """You are a film analyst. You job is analyzing shorts or derivatives of 
films, like promo or trailer. Finding the matching film and identify key cast memebers."""

agent_instruction = """
Your task is to analyze film videos or deriative of film videos and match it to the correct
film title and identify key cast memebers.

Your capabilities include:
<capabilities>
- Matching video to correct film title
- Identify key cast members
</capabilities>

Response style:
<Response style>
- Be helpful and solution-oriented
- Use clear, non-technical language
- Maintain natural conversation flow
- Be concise yet informative
- do not add extra information not required by the user
</Response style>
"""

film_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(10)

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    film_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)
time.sleep(20)

## Test Knowledge Base W/ Agent

Let's test our Knowledge Base with Agent we have just created.

In [ ]:
%%time
import uuid

response = agents.invoke(
    input_text=f"which film is directored by Curtis Clark", 
    agent_id=film_agent[0], 
    enable_trace=True,
    session_id=str(uuid.uuid4()),
)
print("====================")
print(response)

Store the parameters for future labs

In [ ]:
%store knowledge_base_name
%store kb_config
%store agent_instruction
%store agent_name
%store video_analysis

### Creating Lambda

Now you need to create a Lambda function that will be used by the agent to detect celebrities (actors) in video clips, which is essential for identifying films based on the appearance of known cast members.

### Using Amazon Q to explain code

In the cell below there is alot of code, here we can use Amazon Q to help us understand the logic.  Before running the following cell, try highlighting any part of the code below, right click on the highlighted code, and select 'Generative AI->Explain Code'. This will then send the selected code to Amazon Q in the left window and it will provide a detailed explanation on the logic. Amazon Q is very useful for helping developers understand a code base. Feel free to repeat this process on any block of code to help you further understand the logic. 

In [ ]:
%%writefile detection.py
import boto3
import json
import os
import time
from boto3.dynamodb.conditions import Key, Attr

## DynamoDB parameters
dynamodb_resource = boto3.resource('dynamodb')
cast_table = os.getenv('cast_table')
cast_pk = os.getenv('cast_pk')

## Rekognition parameters
rek_client = boto3.client('rekognition')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except StopIteration:
        raise ValueError(f"Required parameter '{name}' not found in event")
        
def get_cast_member(cast_id):
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {cast_table}.')

def start_celebrity_detection(bucket, video_key):
    response = rek_client.start_celebrity_recognition(
        Video={
            'S3Object': {
                'Bucket': bucket,
                'Name': video_key
            }
        }
    )
    return response['JobId']

def get_celebrity_detection_results(job_id):
    response = rek_client.get_celebrity_recognition(JobId=job_id)
    return response

def extract_bucket_key(video_s3_path):
    path = video_s3_path[5:]  # Remove 's3://'
    bucket, key = path.split('/', 1)  # Split into bucket and key   
    return bucket, key

def detect_key_figures(video_s3_path):
    bucket, key = extract_bucket_key(video_s3_path)
    
    job_id = start_celebrity_detection(bucket, key)
    print(f"Started celebrity detection job: {job_id}")

    while True:
        response = get_celebrity_detection_results(job_id)
        status = response['JobStatus']
        
        if status in ['SUCCEEDED', 'FAILED']:
            print("JOB COMPLETE....")
            break
        
        print("Job in progress...")
        time.sleep(10)

    unique_celebrities = {}  # Dictionary to store unique celebrities

    if status == 'SUCCEEDED':
        for celebrity in response['Celebrities']:
            celeb = celebrity['Celebrity']
            
            # Only process celebrities with 95%+ confidence
            if celeb['Confidence'] >= 95.0:
                celeb_id = celeb['Id']
                
                # Store or update celebrity info only if not already stored
                if celeb_id not in unique_celebrities:
                    celebrity_info = {
                        'name': celeb['Name'],
                        'confidence': celeb['Confidence'],
                        'id': celeb_id,
                        'first_appearance': celebrity['Timestamp']
                    }
                    
                    # If you have additional celebrity info in DynamoDB
                    try:
                        query_items = get_cast_member(celeb_id)
                        if query_items:
                            celebrity_info.update(query_items[0])
                    except Exception as e:
                        print(f"Error fetching additional celebrity info: {str(e)}")
                    
                    unique_celebrities[celeb_id] = celebrity_info
    else:
        print("Detection failed....")

    # Convert the dictionary values to a list for the final output
    final_output = list(unique_celebrities.values())
    return final_output

def populate_function_response(event, response_body):
    return {
        'response': {
            'actionGroup': event['actionGroup'],
            'function': event['function'],
            'functionResponse': {
                'responseBody': {
                    'TEXT': {
                        'body': str(response_body)
                    }
                }
            }
        }
    }
    
def lambda_handler(event, context):
    print(event)
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    video_s3_path = get_named_parameter(event, "video_s3_path")

    if function == 'detect_key_figures':
        result = detect_key_figures(video_s3_path)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

You will also need to define a function schema for the celebrity detection action. 

This schema serves as a contract between the agent and the Lambda function, defining what the function does and what inputs it requires. The agent will use this schema to understand when and how to call the function, and to validate that it has all the necessary information before making the call. This structured approach ensures reliable communication between the agent and the Lambda function.

In [ ]:
functions_def = [
    {
        "name": "detect_key_figures",
        "description": """Detect key figures (celebrity, cast member) from a video
        and retrieve information about their position and team""",
        "parameters": {
            "video_s3_path": {
                "description": "S3 location of the video (e.g: s3://......)",
                "required": True,
                "type": "string"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_lambda_action_group_with_rek(
    agent_name=agent_name,
    lambda_function_name=lambda_name,
    source_code_file="detection.py",
    agent_functions=functions_def,
    agent_action_group_name="key_figure_detection_actions",
    agent_action_group_description="Functions to identify key figures and look up cast members and their role from the video",
    env_args=env_args
)
time.sleep(30)

### Loading DynamoDB

The following step populates the DynamoDB table with information about cast members that can be used to enrich the celebrity detection results. When the Lambda function detects a celebrity in a video, it can look up additional information about that person, such as what film they appeared in and what role they played, which is crucial for accurate film identification.

In [ ]:
# Open and read the JSONL file
with open('cast_members.jsonl', 'r') as file:
    table_items = [json.loads(line.strip()) for line in file]

agents.load_dynamodb(cast_table, table_items)

Now let's test the DynamoDB query functionality for cast member lookup !

We will use the specific celebrity ID ('4kn3Xu8r') to look up:

In [ ]:
resp = agents.query_dynamodb(
    cast_table, cast_pk, '4kn3Xu8r'
)
resp

This test confirms that the DynamoDB table is properly populated and can be queried to retrieve cast member information. The Lambda function will use this same query mechanism to enrich celebrity detection results with film and role information, which is essential for the agent to accurately identify films based on the actors that appear in them.

In [ ]:
%store cast_table
%store cast_pk
%store film_video_s3_path

### Test Agent

Here you will be introduced to Bedrock agent session context. It's an object that preserves conversation context and user attributes across interactions, enabling agents to deliver coherent and personalized experiences while also giving developers granular control to modify, manage, and checkpoint session data as needed throughout the workflow. To learn more about [Control agent session context
](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html)

In this example, we will pass following information to the session state to enable precise control of agent behavior.

1. The video analysis data for understanding the content
2. The video S3 path for celebrity detection
3. Access to the knowledge base for film information lookup

By including both the analysis results and the S3 path, the agent can use multiple approaches to identify the film - matching the plot details against the knowledge base and detecting celebrities in the video itself.

In [ ]:
session_state = {
    'promptSessionAttributes': {
        "<video_analysis>": json.dumps(video_analysis),
        "<video_s3_path>": film_video_s3_path,
    },
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

After execution, the agent will perform following actions:
   - Step 1: Analyzing the request and planning the approach
   - Step 2: Searching the knowledge base for films matching the plot details
   - Step 3: Deciding to verify with celebrity detection
   - Step 4: Calling the celebrity detection function and analyzing results
   - Final response: Shows the agent's conclusion that the clip is from "Meridian" (2016), confirmed by both plot details and the appearance of actors Kevin Kilner and Reid Scott.

This test demonstrates the agent's ability to combine multiple sources of information - knowledge base search and celebrity detection - to accurately identify a film from a video clip. The execution trace shows the agent's reasoning process, making the identification process transparent and explainable.
<div class="alert-warning">
    <b>Warning:</b> Because of rekignition video API for celebrity detection, This test may take up to 7 minutes to complete.
</div>

In [ ]:
%%time
response = agents.invoke(
    input_text=f"""
    Here is a clip extraction:
    $prompt_session_attributes.video_analysis$

    if needed, here is the s3 location of the video:
    $prompt_session_attributes.video_s3_path$
    
    can you tell me which film is this clip from?
    """, 
    agent_id=film_agent[0],
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

### Create alias
Bedrock Agent alias enables seamless version management and deployment flexibility for Amazon Bedrock agents: by assigning an alias (such as "prod" or "dev") to a specific agent version, you can easily route traffic to different agent iterations without changing application code or endpoints.

In [ ]:
film_agent_alias_id, film_agent_alias_arn = agents.create_agent_alias(
    film_agent[0], 'v1'
)

### Clean Up
<div class="alert-info">
    <b>Info:</b> If at any point of this module, you need to re-run a resource creation. You can use the code below to delete existing resurce first.
</div>

In [ ]:
# agents.delete_agent(agent_name)

In [ ]:
# kb.delete_kb(knowledge_base_name)

In [ ]:
# agents.delete_lambda(lambda_function_name=lambda_name)